In [1]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
# %matplotlib inline

path = "/gpfs/bsc_home/xpliu/inference/vision/classification_and_detection/output-exp18/"
all_files = glob.glob(path + "*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
df.head()

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,16,1,offline,10,10,106.588945,111.375800,108.486432,181.159946,206.093318,218.892548,228.917484,230.497341
1,resnet,none,16,2,offline,10,10,169.631761,72.519925,72.046130,114.845634,129.288536,136.337987,142.034885,144.834119
2,resnet,none,16,4,offline,10,10,126.946604,94.857626,94.649063,151.452697,170.965201,180.931538,189.190537,193.226428
3,resnet,none,16,8,offline,10,10,120.825122,100.093711,99.749260,159.845325,179.797115,189.775811,197.402195,202.608210
4,resnet,none,16,16,offline,10,10,144.765961,82.057759,81.174013,131.610065,148.699759,157.365670,164.026586,168.841285


In [2]:
len(df)

10

In [3]:
df['benchmark'].unique()

array(['resnet'], dtype=object)

In [4]:
df['k8senv'].unique()

array(['none', 'cpumem'], dtype=object)

In [5]:
type(df['qps'][0])

numpy.float64

# Graph

In [6]:
import pandas as pd
import glob
# import plotly.plotly as py
import plotly.graph_objs as go

from plotly import tools
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True) 

import numpy as np
import matplotlib.pyplot as plt
# include if using a Jupyter notebook
%matplotlib inline
dfg = df
dfg

,benchmark,k8senv,sbatch,numctn,cscenario,cbatch,reps,qps,mean,p50,p80,p90,p95,p99,p999
0,resnet,none,16,1,offline,10,10,106.588945,111.375800,108.486432,181.159946,206.093318,218.892548,228.917484,230.497341
1,resnet,none,16,2,offline,10,10,169.631761,72.519925,72.046130,114.845634,129.288536,136.337987,142.034885,144.834119
2,resnet,none,16,4,offline,10,10,126.946604,94.857626,94.649063,151.452697,170.965201,180.931538,189.190537,193.226428
3,resnet,none,16,8,offline,10,10,120.825122,100.093711,99.749260,159.845325,179.797115,189.775811,197.402195,202.608210
4,resnet,none,16,16,offline,10,10,144.765961,82.057759,81.174013,131.610065,148.699759,157.365670,164.026586,168.841285
5,resnet,cpumem,16,1,offline,10,10,107.604063,111.534993,109.961001,180.817439,204.806096,217.106372,226.674239,228.298604
6,resnet,cpumem,16,2,offline,10,10,185.892883,66.628415,66.606731,105.979862,119.073703,125.730141,130.915794,132.155354
7,resnet,cpumem,16,4,offline,10,10,187.544177,66.139847,66.473618,105.143584,118.225472,124.369730,129.566874,131.004091
8,resnet,cpumem,16,8,offline,10,10,183.499651,66.757909,66.630437,106.125194,119.411115,125.829775,131.242618,133.459592
9,resnet,cpumem,16,16,offline,10,10,178.500238,67.649300,67.288980,107.806722,121.074165,127.705237,133.078990,136.720635


In [8]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from svglib.svglib import svg2rlg
from reportlab.graphics import renderPDF
from reportlab.pdfgen import canvas

from numpy import percentile, mean, std
import plotly.graph_objects as go

from plotly.offline import init_notebook_mode, iplot

import plotly.io as pio
pio.templates.default = "none"
# pio.templates.default = "simple_white"

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd
# import plotly.express as px

for x in ['qps','mean','p99']:
    for cbatch in [10]:
        for model_name in ['resnet']:
            list_traces = []
            for numctn in [1,2,4,8,16,32]: 
                dfg_bench = dfg[(dfg['benchmark']== model_name)  & (dfg['cbatch']==cbatch) & (dfg['numctn']==numctn) &\
                            ((dfg['k8senv']=='none') |\
                             (dfg['k8senv']=='cpumem')| (dfg['k8senv']=='cpumemory'))]
#                 print(dfg_bench)


                #data
                trace = go.Scatter(
                    name="ctn"+str(numctn),
                    x=[ 'none','cpumem','cpu'],
                    y= dfg_bench[x].values.tolist(),
        #             error_y = dict(type='data', array=df1_bench['model_time_std'].values.tolist()),
                    opacity=0.7,
                    mode='markers'
                )
                list_traces.append(trace)



            #layout
            layout = go.Layout(
                title={
                    'text': "exp18(cpu32)= inter16,intra32 -> inter1,intra2; restapi=32" ,
                    'y':0.85,
                    'x':0.5,
                    'xanchor': 'center',
                    'yanchor': 'top'},
                width=1000,
                height=460,
                xaxis=dict(
                    title= "scenarios",
                    tickvals=[ 'none','cpumem','cpu'],
                    titlefont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=10,
                        color='rgb(107, 107, 107)'
                    )
                ),
                yaxis=dict(
                    title= x+"batch"+str(cbatch),
                    automargin=True,
                    titlefont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),
                    tickfont=dict(
                        size=25,
                        color='rgb(107, 107, 107)'
                    ),    
                ),
                legend=dict(
                    x=1.0,
                    y=1.0,
                    bgcolor='rgba(255, 255, 255, 0)',
                    bordercolor='rgba(255, 255, 255, 0)',
                    font = dict( size = 28)
                ),
                barmode='group',
                bargap=0.15,
                bargroupgap=0.08
        #         legend_orientation="h"
            )

        data = list_traces
        fig = go.Figure(data=data, layout=layout)
        fig.update_layout(legend_orientation="h")
        fig.show()    

    #     fig.write_image(path + model_name + k8senv +'.pdf')